In [1]:
import pandas as pd
import numpy as np

In [2]:
funnel = pd.read_csv("funnel_step_metrics.csv")
checkout = pd.read_csv("checkout_user_segments.csv")

In [3]:
checkout.head()

,user_pseudo_id,device_category,begin_checkout_count,purchase_count,total_revenue,checkout_segment
0,7.044248e+07,mobile,1,0,0,Other Checkout Users
1,2.864874e+07,desktop,1,0,0,Other Checkout Users
2,3.464936e+07,desktop,1,0,0,Other Checkout Users
3,7.181362e+06,mobile,1,0,0,Other Checkout Users
4,8.517561e+06,desktop,1,0,0,Other Checkout Users


In [4]:
funnel.head()

,step_order,step,users,conversion_rate
0,1,Viewed Item,62576,1.000000
1,2,Add to Cart,12796,0.204487
2,3,Begin Checkout,9770,0.763520
3,4,Purchase,4461,0.456602


In [5]:
checkout["total_revenue"].describe()

count    9852.000000
mean       36.760556
std        83.449205
min         0.000000
25%         0.000000
50%         0.000000
75%        46.000000
max      1530.000000
Name: total_revenue, dtype: float64

In [6]:
# total checkout users
total_checkout_users = checkout.shape[0]
print(f"Total Checkout Users = {total_checkout_users}" )

# repeated failure users
repeated_failures = checkout[
    checkout["checkout_segment"] == "Repeated Checkout Failure"
].shape[0]
print(f"Repeated Failures = {repeated_failures}")

failure_rate = repeated_failures / total_checkout_users
print(f"Failure rate = {failure_rate}")

Total Checkout Users = 9852
Repeated Failures = 1770
Failure rate = 0.17965895249695493


In [7]:
avg_order_value = checkout.loc[
    checkout["purchase_count"] >= 1, "total_revenue"
].mean()

avg_order_value

np.float64(80.67832479394075)

In [8]:
estimated_revenue_at_risk = repeated_failures * avg_order_value
estimated_revenue_at_risk

np.float64(142800.63488527513)

In [9]:
device_summary = (
    checkout
    .groupby("device_category")
    .agg(
        checkout_users=("user_pseudo_id", "count"),
        repeated_failures=("checkout_segment", lambda x: (x == "Repeated Checkout Failure").sum())
    )
    .reset_index()
)

device_summary["failure_rate"] = (
    device_summary["repeated_failures"] / device_summary["checkout_users"]
)

device_summary["estimated_revenue_at_risk"] = (
    device_summary["repeated_failures"] * avg_order_value
)

device_summary


,device_category,checkout_users,repeated_failures,failure_rate,estimated_revenue_at_risk
0,desktop,5665,1060,0.187114,85519.024282
1,mobile,3968,672,0.169355,54215.834262
2,tablet,219,38,0.173516,3065.776342


In [10]:
device_summary.to_csv(
    "device_level_checkout_summary.csv",
    index=False
)